# `ipydatagrid`: fast, performant data grid

## https://github.com/bloomberg/ipydatagrid


`ipydatagrid` is a complete data grid solution providing full integration with pandas DataFrames without compromising on performance:

- built-in sorting and filtering
- full integration with the jupyter-widgets ecosystem
- highly customisable renderers
- complete two-way data binding between Python and the front-end
- supports Vega expressions for conditional formatting and styling

**Installation**:
```bash
conda install -c conda-forge ipydatagrid
```

In [1]:
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
import pandas as pd
import numpy as np
import requests

#### Use any dataset

In [2]:
# Numpy/pandas
np.random.seed(0)
p_t, n = 100, 260
stock_df = pd.DataFrame(
    {f'Stock {i}': p_t + np.round(np.random.standard_normal(n).cumsum(), 2) for i in range(10)}
)

# Requests/JSON
req = requests.get("https://raw.githubusercontent.com/bloomberg/ipydatagrid/main/examples/cars.json")
data = req.json()
cars_data = data['data']


# Random matrix
rand_df = pd.DataFrame(
    {f'Column {col}': np.round(np.random.random(100), 2) for col in [chr(n) for n in range(65, 91)]}
)

# Small grid
small_df = pd.DataFrame(
    np.eye(10),
    columns=[f'Col {i}' for i in range(10)],
    index=[f'Row {i}' for i in range(10)]
)

# Multi-index
top_level = ['Value Factors', 'Value Factors', 'Momentum Factors', 'Momentum Factors']
bottom_level = ['Factor A', 'Factor B', 'Factor C', 'Factor D']

nested_df = pd.DataFrame(np.random.randn(4,4).round(2),
                         columns=pd.MultiIndex.from_arrays([top_level, bottom_level]),
                         index=pd.Index(['Security {}'.format(x) for x in ['A', 'B', 'C', 'D']], name='Ticker'))

#### Convert your pandas DataFrame to a datagrid - it 'just works'

In [3]:
df = pd.DataFrame(cars_data).set_index('index')
df.head()

,Horsepower,Origin,Miles_per_Gallon,Name,Acceleration,Year,Weight_in_lbs,Cylinders,Displacement
index,,,,,,,,,
0,130.0,USA,18.0,chevrolet chevelle malibu,12.0,1970-01-01,3504,8,307.0
1,165.0,USA,15.0,buick skylark 320,11.5,1970-01-01,3693,8,350.0
2,150.0,USA,18.0,plymouth satellite,11.0,1970-01-01,3436,8,318.0
3,150.0,USA,16.0,amc rebel sst,12.0,1970-01-01,3433,8,304.0
4,140.0,USA,17.0,ford torino,10.5,1970-01-01,3449,8,302.0


In [4]:
grid = DataGrid(df)
grid

/home/idafna/miniconda3/envs/widgets-tutorial-2021/lib/python3.8/site-packages/pandas/io/json/_table_schema.py:97: UserWarning: Index name of 'index' is not round-trippable
  warnings.warn("Index name of 'index' is not round-trippable")


DataGrid(default_renderer=TextRenderer(), header_renderer=None)

Additional constructor options

In [5]:
grid = DataGrid(
    dataframe=df,
    base_row_size=30,
    base_column_size=92,
    base_row_header_size=128,
    base_column_header_size=40
)
grid

DataGrid(base_column_header_size=40, base_column_size=92, base_row_header_size=128, base_row_size=30, default_…

In [6]:
grid.header_visibility = 'row'

In [7]:
grid.header_visibility = 'column'

In [8]:
grid.header_visibility = 'none'

In [9]:
grid.header_visibility = 'all'

#### Style your data grid with custom renderers based on Vega expressions

- Custom cell renderers can be defined for the entire grid or column-wise.

- Two types of cell renderers are currently available: `TextRenderer` and `BarRenderer`.

- Most of the TextRenderer/BarRenderer attributes (`background_color`, `text_color` etc.) can either be a `value`, a `bqplot` scale or a `VegaExpr` or `Expr` instance.

The `VegaExpr` class allows you to define an attribute value as a result of a Vega-expression (see https://vega.github.io/vega/docs/expressions/):
```python
background_color = VegaExpr("value < 150 ? 'red' : 'green'").
```
You can look at the vega-expression documentation for more information about available constants and functions. In the scope of the expression are also available: value: cell value, x and y: cell position in pixel, width and height of the cell, row and column: cell position.

In [10]:
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from py2vega.functions.color import rgb

def horsepower_coloring(cell):
    if cell.value < 100:
        return "red"
    elif cell.value < 150:
        return "orange"
    else:
        return "green"


def weight_coloring(cell):
    scaled_value = 1 if cell.value > 4500 else cell.value / 4500
    color_value = scaled_value * 255

    return rgb(color_value, 0, 0)


renderers = {
    "Acceleration": BarRenderer(
        horizontal_alignment="center",
        bar_color=ColorScale(min=0, max=20, scheme="viridis"),
        bar_value=LinearScale(min=0, max=20),
    ),
    "Cylinders": TextRenderer(
        background_color=Expr('"grey" if cell.row % 2 else default_value')
    ),
    "Displacement": TextRenderer(
        text_color=ColorScale(min=97, max=455),
        font=Expr(
            "'16px sans-serif' if cell.value > 400 else '12px sans-serif'"
        ),
    ),
    "Horsepower": TextRenderer(
        text_color="black", background_color=Expr(horsepower_coloring)
    ),
    "Miles_per_Gallon": TextRenderer(
        background_color=Expr('"grey" if cell.value is None else default_value')
    ),
    "Name": TextRenderer(
        background_color=Expr(
            'rgb(0, 100, 255) if "chevrolet" in cell.value or "ford" in cell.value else default_value'
        )
    ),
    "Origin": TextRenderer(
        text_color="black",
        background_color=OrdinalColorScale(domain=["USA", "Japan", "Europe"]),
        horizontal_alignment=Expr(
            "'right' if cell.value in ['USA', 'Japan'] else 'left'"
        ),
    ),
    "Weight_in_lbs": TextRenderer(
        text_color="black", background_color=Expr(weight_coloring)
    ),
    "Year": TextRenderer(text_color="black", background_color="green"),
}

grid.renderers = renderers

In [11]:
renderers["Name"] \
    .background_color.value = '"green" if "pontiac" in cell.value or "citroen" in cell.value else default_value'

In [12]:
renderers["Year"].background_color = "yellow"

#### Apply styling to column and row headers

In [13]:
grid.header_renderer = TextRenderer(
    background_color=Expr('"salmon" if cell.value == "Horsepower" else "skyblue"'),
    font='italic small-caps bold 12px/30px Georgia, serif'
)

renderers['index'] = TextRenderer(background_color='slateblue')
grid.renderers = renderers

#### Built-in sorting and filtering functionality which you can trigger from both Python and directly via the GUI.

In [14]:
# Filtering based on car origin and sorting based on displacement
grid.transform(
    [
        {
            "type": "filter",
            "operator": "=",
            "columnIndex": 2,
            "value": "Europe",
        },
        {"type": "sort", "columnIndex": 9, "desc": True},
    ]
)

#### Conditional formatting based on another cell

In [15]:
def format_based_on_other_column(cell):
    return ("green" if cell.column == 2 and cell.metadata.data["Return"] > 0 else "red")

signal_column_formatting = TextRenderer(
    text_color="white",
    background_color=Expr(format_based_on_other_column),
)

renderers = {
    "Signal": signal_column_formatting,
    "Return": TextRenderer(background_color='seashell',
                           text_color=VegaExpr('cell.value > 0 ? "green" : "firebrick"')
    )
}

conditional_grid = DataGrid(
    pd.DataFrame(
        {"Stock": "A B C D".split(), 
         "Return": [0.11, -0.05, 0.08, -0.20], 
         "Signal": ["Buy", "Sell", "Buy", "Sell"]}
    ),
    column_widths={"Stock": 64, "Return": 64,  "Signal": 300},
    base_row_size=30,
    renderers=renderers,
    layout={"height": "150px"},
)

conditional_grid

DataGrid(base_row_size=30, column_widths={'Stock': 64, 'Return': 64, 'Signal': 300}, default_renderer=TextRend…

#### Multi-index and nested columns DataFrames are also supported

In [16]:
nested_df

Value Factors          Momentum Factors         
                Factor A Factor B         Factor C Factor D
Ticker                                                     
Security A         -0.76    -1.53             1.16    -0.51
Security B         -0.45    -1.39             0.53     0.57
Security C         -0.55    -1.19            -1.89     0.57
Security D          2.60     0.40             0.61     0.15

In [17]:
columns_renderer = TextRenderer(
    background_color='dimgray',
    horizontal_alignment='center')

renderers= {
    "('Ticker', '')": TextRenderer(background_color='dimgray')
}

default_renderer = TextRenderer(
    background_color=VegaExpr('cell.value > 0 ? "steelblue" : "seagreen"')
)

nested_grid = DataGrid(nested_df,
                       base_column_size=90,
                       column_widths={"('Ticker', '')": 80},
                       layout={'height':'140px'},
                       renderers=renderers,
                       default_renderer=default_renderer,
                       header_renderer=columns_renderer)

nested_grid

DataGrid(base_column_size=90, column_widths={"('Ticker', '')": 80}, default_renderer=TextRenderer(background_c…

#### Two-way selections model at your disposal

DataGrid cells can be selected using mouse by simply clicking and dragging over the cells. Pressing Cmd / Ctrl key during selection will add to existing selections. Pressing Shift key allows selection of regions between two clicks.

DataGrid supports three modes of selection `cell`, `row`, `column`. In order to disable selections, selection mode can be set to `none` which is the default setting.

Selection Modes:

- `cell`: Clicking on grid will select only the cell under mouse cursor
- `row`: Clicking on grid will select all the cells on the row under mouse cursor
- `column`: Clicking on grid will select all the cells on the column under mouse cursor

You can clear all selections by hitting the `esc` key.

In [18]:
sel_grid = DataGrid(stock_df, selection_mode='cell')
sel_grid

DataGrid(default_renderer=TextRenderer(), header_renderer=None, selection_mode='cell')

Select from the UI and retrieve on the Python side

In [19]:
sel_grid.selections

[]

We can also access the selected cell values

In [20]:
sel_grid.selected_cell_values

[]

Select from the Python side and see selections highlighted on the UI.

Parameters:

- `row1`: start row (starts from 0).
- `row2`: end row (starts from 0).
- `column1`: start column.
- `column2`: end column.

We can automatically clear any existing selections by passing a value for `clear_mode`:

- `current`: clear last selection
- `all`: clear all existing selections
- `none`: do not clear selections (default)

In [21]:
# Select top left corner of grid
sel_grid.select(row1=0, column1=0, row2=1, column2=1, clear_mode='current')

In [22]:
# Clear selection
sel_grid.clear_selection()

We can select individual sells by omitting `row2` and `column2`

In [23]:
sel_grid.select(10, 5)

When working with large grids, we can opt to use the `selected_cell_iterator`. It will yield values for each loop iteration, avoiding the need to store all selections in a list, in advance.

In [24]:
for cell in sel_grid.selected_cell_iterator:
    print(f'Cell value: {cell}')

Cell value: {'r': 10, 'c': 5}


We can modify selections in place by passing a list of selections

In [25]:
sel_grid.selections = [
    {"r1": 22, "r2": 20, "c1": 0, "c2": 2},
    {"r1": 6, "r2": 6, "c1": 2, "c2": 2},
    {"r1": 10, "r2": 10, "c1": 3, "c2": 3},
    {"r1": 13, "r2": 13, "c1": 2, "c2": 2},
]

Row selection mode

In [26]:
sel_grid.selection_mode='row'

In [27]:
for i in range(260):
    sel_grid.select(i, i) if i % 2 == 0 else None

#### Two-way cell editing is possible

Just pass `editable=True` to the grid's constructorand you're good to go (grids are not editable by default).

In [28]:
small_grid = DataGrid(small_df, 
                      editable=True, 
                      default_renderer=TextRenderer(
                          background_color=VegaExpr("cell.value === 1 ? 'limegreen' : 'hotpink'")
                      ),
                      layout={'height': '250px'})
small_grid

DataGrid(default_renderer=TextRenderer(background_color=VegaExpr(value="cell.value === 1 ? 'limegreen' : 'hotp…

You can change values directly in the UI by double clicking a cell and changing the value. You can navigate the grid using the keyboard. You can use the arrow keys or the grid's __cursor__:

- __Down__: Enter
- __Up__: Shift + Enter
- __Right__: Tab
- __Left__: Shift + Tab

..or you can change it directly from the Python side

In [29]:
# Returns a boolean to indicate whether operations was successful
small_grid.set_cell_value('Col 0', 'Row 9', 1) # Sets value based on row name

True

In [30]:
small_grid.set_cell_value_by_index('Col 9', 0, 1) # Sets value based on row index

True

#### Events and integration with `ipywidgets`

Listen to cell change events

In [31]:
def cell_changed(e):
    row, column, col_index, value = e['row'], e['column'], e['column_index'], e['value']
    print(f'The cell at row {row}, column "{column}" (index {col_index}), changed to {value}')
    
small_grid.on_cell_change(cell_changed)
small_grid

DataGrid(default_renderer=TextRenderer(background_color=VegaExpr(value="cell.value === 1 ? 'limegreen' : 'hotp…

#### An example with the BarRenderer

Renders cell values as horizontal bars based on a scale. `ipydatagrid` has two renderers - `TextRenderer`, which is the default one we've seen, and `BarRenderer`, which we will use now.

In [32]:
from bqplot import LinearScale, ColorScale
from ipydatagrid import DataGrid, BarRenderer

linear_scale = LinearScale(min=0, max=1)
color_scale = ColorScale(min=0, max=1)
bar_renderer = BarRenderer(
    bar_color=color_scale,
    bar_value=linear_scale,
    bar_horizontal_alignment="center",
)


rand_grid = DataGrid(rand_df, default_renderer=bar_renderer, base_column_size=76)
rand_grid

DataGrid(base_column_size=76, default_renderer=BarRenderer(bar_color=ColorScale(max=1.0, min=0.0), bar_horizon…

In [33]:
bar_renderer.show_text = False

In [34]:
from ipywidgets import FloatSlider, link

slider = FloatSlider(
    description="Scale: ", value=linear_scale.max, min=0, max=0.99, step=0.01
)
link((color_scale, "min"), (slider, "value"))
link((linear_scale, "min"), (slider, "value"))

slider

FloatSlider(value=0.0, description='Scale: ', max=0.99, step=0.01)

In [35]:
color_scale.min

0.0

#### Integration with `bqplot`

We have a DataGrid with time series of 10 different stock prices. Each time we click on a stock price column, we want to plot the time series of that stock price in a line chart.

In [36]:
stock_df.head()

,Stock 0,Stock 1,Stock 2,Stock 3,Stock 4,Stock 5,Stock 6,Stock 7,Stock 8,Stock 9
0,101.76,98.81,99.78,101.49,101.67,98.55,101.59,99.23,100.33,100.55
1,102.16,98.30,98.86,102.01,101.97,98.10,100.94,98.96,100.45,100.55
2,103.14,97.71,97.18,102.62,102.58,98.42,99.81,99.78,99.44,99.01
3,105.38,97.66,96.29,101.28,103.69,98.28,100.81,99.48,98.41,99.87
4,107.25,95.72,96.54,101.76,105.13,97.33,100.66,98.56,98.73,99.75


In [102]:
from bqplot import LinearScale, Axis, Figure, Lines, CATEGORY10
from ipywidgets import HBox, Layout
    
# Setting up the data grid
stock_grid = DataGrid(stock_df, selection_mode='column')

# Creating the bqplot chart objects
sc_x = LinearScale()
sc_y = LinearScale()
line = Lines(x=[], y=[], labels=['Fake stock price'], display_legend=True,
                 scales={'x': sc_x, 'y': sc_y})
ax_x = Axis(scale=sc_x, label='Index')
ax_y = Axis(scale=sc_y, orientation='vertical', label='y-value')
fig = Figure(marks=[line], axes=[ax_x, ax_y], title='Line Chart', layout=Layout(flex='1 1 auto', width='100%'))

In [103]:
def plot_stock(*args):
    line.y = stock_grid.selected_cell_values
    line.x = range(len(line.y))
    column_index = stock_grid.selections[0]['c1']
    line.labels = [stock_df.columns[column_index]]
    line.colors = [CATEGORY10[np.random.randint(0, len(CATEGORY10)) % len(CATEGORY10)]]
    
# Event listener for cell click
stock_grid.on_msg(plot_stock)

In [104]:
stock_grid.observe(plot_stock, names='selections')

In [105]:
HBox(
    [stock_grid, fig]
)